# 🌊 FlowGrad Demo Notebook

**One-line Training Diagnostics & Feature Engineering for DL, ML, and RecSys**

이 노트북에서는 FlowGrad의 모든 핵심 기능을 실제 데이터로 테스트합니다:
1. 🔧 **sklearn** - GradientBoosting + RandomForest 학습 추적
2. 🌲 **XGBoost** - 부스팅 라운드별 진단
3. 🧪 **Feature Engineering** - 피처 상호작용·조합 제안·중복 탐지
4. 🔬 **PyTorch** - DL 레이어별 건강 추적

## 0. Installation

In [ ]:
# GitHub에서 바로 설치 (YOUR_USERNAME을 실제 GitHub 유저명으로 바꾸세요)
!pip install git+https://github.com/YOUR_USERNAME/FlowGrad.git"[all]" -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer, fetch_california_housing
from sklearn.model_selection import train_test_split

print("✅ Setup complete!")

---
## 1. 🔧 sklearn — GradientBoosting 학습 추적

**데이터**: Breast Cancer (sklearn 내장, 569 samples, 30 features, binary classification)

GradientBoosting의 `warm_start`를 활용해 10라운드씩 학습하면서 피처 중요도 변화를 추적합니다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from flowgrad import SklearnTracker

# 데이터 준비
data = load_breast_cancer()
X, y = data.data, data.target
feature_names = list(data.feature_names)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Features: {len(feature_names)}, Train: {len(X_train)}, Val: {len(X_val)}")
print(f"Feature names: {feature_names[:5]}...")

In [ ]:
# ✨ FlowGrad 한 줄 추적
tracker = SklearnTracker(feature_names=feature_names)
model = GradientBoostingClassifier(n_estimators=100, max_depth=3, warm_start=True, random_state=42)
tracker.track_warm_start(model, X_train, y_train, X_val, y_val, step_size=10)

print(tracker)

In [ ]:
# 📊 종합 진단 리포트
tracker.report();

In [ ]:
# 📈 피처 중요도 변화 시각화
fig = tracker.plot.feature_drift(top_k=8)
plt.show()

In [ ]:
# 🔥 피처 중요도 히트맵
fig = tracker.plot.feature_importance_heatmap(top_k=12)
plt.show()

In [ ]:
# ⚠️ 과적합 탐지
fig = tracker.plot.overfitting_detector()
plt.show()

In [ ]:
# 📋 종합 대시보드 (4-panel)
fig = tracker.plot.full_report()
plt.show()

---
## 2. 🌲 RandomForest — 개별 트리 분석

RandomForest의 각 트리가 어떤 피처를 중요시하는지 분석합니다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
rf_model.fit(X_train, y_train)

# ✨ 트리별 피처 중요도 분석
rf_tracker = SklearnTracker.from_forest(rf_model, feature_names=feature_names)
print(rf_tracker)
print(f"\nAccuracy: {rf_model.score(X_val, y_val):.4f}")

In [ ]:
# 트리별 피처 중요도 변화
fig = rf_tracker.plot.feature_drift(top_k=6)
plt.show()

---
## 3. 🌲 XGBoost — 부스팅 라운드별 진단

**데이터**: California Housing (sklearn 내장, 20640 samples, 8 features, regression)

In [ ]:
import xgboost as xgb
from flowgrad import BoostingTracker

# 데이터 준비
housing = fetch_california_housing()
X_h, y_h = housing.data, housing.target
feat_names_h = list(housing.feature_names)
X_h_train, X_h_val, y_h_train, y_h_val = train_test_split(X_h, y_h, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_h_train, label=y_h_train, feature_names=feat_names_h)
dval = xgb.DMatrix(X_h_val, label=y_h_val, feature_names=feat_names_h)

print(f"Features: {feat_names_h}")
print(f"Train: {len(X_h_train)}, Val: {len(X_h_val)}")

In [ ]:
# ✨ FlowGrad 한 줄 추적
xgb_tracker = BoostingTracker()

params = {
    "objective": "reg:squarederror",
    "max_depth": 6,
    "learning_rate": 0.1,
    "subsample": 0.8,
}

xgb_model = xgb.train(
    params, dtrain,
    num_boost_round=200,
    evals=[(dtrain, "train"), (dval, "valid")],
    callbacks=[xgb_tracker.as_xgb_callback()],
    verbose_eval=False,
)

print(xgb_tracker)

In [ ]:
# 📊 종합 진단 리포트
xgb_tracker.report();

In [ ]:
# 📈 평가 메트릭 + 피처 중요도 변화
fig = xgb_tracker.plot.eval_metrics()
plt.show()

fig = xgb_tracker.plot.feature_drift(top_k=8)
plt.show()

In [ ]:
# ⚠️ 과적합 탐지 + 종합 대시보드
fig = xgb_tracker.plot.overfitting_detector()
plt.show()

fig = xgb_tracker.plot.full_report()
plt.show()

---
## 4. 🧪 Feature Engineering — 피처 상호작용 분석 ⭐

**FlowGrad의 핵심 차별화 기능!**

기존 `df.corr()`이나 `model.feature_importances_`와는 다르게:
- 비선형 피처 상호작용(시너지)을 측정하고
- 새로운 피처 조합(A*B, A/B 등)을 자동으로 제안합니다.

In [ ]:
from flowgrad import FeatureAnalyzer
from sklearn.ensemble import RandomForestRegressor

# California Housing 데이터로 분석
rf_reg = RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42)
rf_reg.fit(X_h_train, y_h_train)

analyzer = FeatureAnalyzer(rf_reg, X_h_train, y_h_train, feature_names=feat_names_h)
print(f"Model R² (val): {rf_reg.score(X_h_val, y_h_val):.4f}")

In [ ]:
# 🔗 피처 상호작용 분석 (어떤 피처 쌍이 시너지가 있는지)
interactions = analyzer.interactions(top_k=10, method="correlation")
print("\n🔗 Top Feature Interactions:")
for i, item in enumerate(interactions, 1):
    print(f"  {i}. {item['feat_a']:>12s} × {item['feat_b']:<12s}  synergy={item['synergy_score']:+.4f}")

In [ ]:
# 💡 피처 조합 제안 (어떤 조합을 만들면 성능이 올라갈까?)
suggestions = analyzer.suggest_features(top_k=10)
print("\n💡 Suggested Feature Combinations:")
for i, item in enumerate(suggestions, 1):
    if item['lift'] > 0:
        print(f"  {i}. CREATE: {item['expression']:<35s}  corr={item['target_correlation']:.4f}  lift=+{item['lift']:.4f}")

In [ ]:
# 🗑️ 중복 피처 탐지
redundant = analyzer.redundant_features(threshold=0.90)
print("\n🗑️ Redundant Features:")
if redundant:
    for item in redundant:
        print(f"  {item['feat_a']} ↔ {item['feat_b']}  corr={item['correlation']:.4f}")
        print(f"  → {item['recommendation']}")
else:
    print("  No redundant features detected at threshold 0.90")

In [ ]:
# 📦 피처 클러스터링
clusters = analyzer.feature_clusters()
print("\n📦 Feature Clusters:")
for cluster in clusters:
    print(f"  Cluster {cluster['cluster_id']} (cohesion={cluster['cohesion_score']:.2f}): {cluster['features']}")

In [ ]:
# 📊 종합 Feature Engineering 리포트
analyzer.report();

In [ ]:
# 🎨 시각화
fig = analyzer.plot.interaction_heatmap(top_k=8)
plt.show()

fig = analyzer.plot.suggestion_chart(top_k=8)
plt.show()

fig = analyzer.plot.cluster_map()
plt.show()

---
## 5. 🔬 PyTorch — DL 레이어별 건강 추적

간단한 MLP 모델을 학습하면서 FlowGrad로 레이어 건강 상태를 실시간 추적합니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from flowgrad import FlowTracker

# Breast Cancer 데이터를 PyTorch로 변환
X_tensor = torch.FloatTensor(X_train)
y_tensor = torch.FloatTensor(y_train).unsqueeze(1)

# 모델 정의
model = nn.Sequential(
    nn.Linear(30, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid(),
)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✨ FlowGrad 한 줄 추적
tracker = FlowTracker(model)
print(tracker)

In [ ]:
# 학습 루프
for epoch in range(50):
    optimizer.zero_grad()
    output = model(X_tensor)
    loss = criterion(output, y_tensor)
    loss.backward()
    optimizer.step()
    
    # ✨ 이것만 추가하면 끝!
    tracker.step(loss=loss.item())

print(f"Final loss: {loss.item():.4f}")
print(tracker.summary)

In [ ]:
# 📊 종합 진단 리포트
tracker.report();

In [ ]:
# 📉 Loss 곡선
fig = tracker.plot.loss()
plt.show()

In [ ]:
# 🔥 Velocity Heatmap (어떤 레이어가 빠르게 학습되는지)
fig = tracker.plot.velocity_heatmap()
plt.show()

In [ ]:
# 📊 Gradient Flow (레이어별 그라디언트 크기)
fig = tracker.plot.gradient_flow()
plt.show()

In [ ]:
# 🏥 Health Dashboard
fig = tracker.plot.health_dashboard()
plt.show()

In [ ]:
# 📡 Gradient SNR
fig = tracker.plot.gradient_snr()
plt.show()

In [ ]:
# 📋 종합 대시보드 (6-panel)
fig = tracker.plot.full_report()
plt.show()

In [ ]:
# 정리
tracker.detach()
print("\n✅ FlowGrad demo complete!")

---
## Summary

| 기능 | 코드 |
|---|---|
| DL 추적 | `tracker = FlowTracker(model)` → `tracker.step(loss=...)` |
| XGBoost 추적 | `callbacks=[tracker.as_xgb_callback()]` |
| sklearn 추적 | `tracker.track_warm_start(model, X, y)` |
| 피처 분석 | `analyzer = FeatureAnalyzer(model, X, y)` |
| 진단 리포트 | `tracker.report()` |
| 시각화 | `tracker.plot.full_report()` |

**GitHub**: [https://github.com/YOUR_USERNAME/FlowGrad](https://github.com/YOUR_USERNAME/FlowGrad)